# Standard Features
## Working with single measurement scans

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib,lmfit
import matplotlib.pyplot as plt
import KiMoPack.plot_func as pf
from importlib import reload
%matplotlib tk

This notebook is an introduction to transient absorption spectroscopy. <br>
In contrast to the other tutorials we are using artificial data for this analysis. <br> 
For this training we will be using 'con_1.SIA' which is clean data with some added noise.<br>
The following dataset 'con_1.SIA' was generated by creating a reaction of the type A->B->C and spectra as shown in this image:
![Chirp](img/Intro_tutorial.png "Data_content")

In [ ]:
filenames =['con_1.SIA','con_2.SIA','con_3.SIA','con_4.SIA']               # set name of the file to fit
#If the path is only one folder one can just give the function the name, otherwise one has to give the absolute path
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')  # set path to file to fit
ta=pf.TA(filenames[0],path=filepath)
plt.close('all')
ta.Plot_RAW(0)

In [ ]:
# Can't remember the commands?
ta()

We investigate the 2D matrix using the option to click on the data. We plot this spectrum twice, once in logaritmic scale and once in linear scale. Making the plot clickable means that the values where you click are shown. 
I usually use this method to define where I do would like to show the plots.

In [ ]:
#the 2d Matrix is plot 0 (check ta.Plot_RAW? for a tutorial)
ta.log_scale=False # This is the default
ta.Plot_RAW(0,print_click_position=True)
ta.log_scale=True   # This scales the 2d plot into log scale.
ta.Plot_RAW(0,print_click_position=True)

Setting these values I plot the same matrix again and select the interesting wavelength using the same process

In [ ]:
ta.bordercut=[400,975]
ta.timelimits=[-0.2,500]
plt.close('all')
ta.Plot_RAW(0,print_click_position=True)

Based on the shown wavelength I set the interesting wavelength and plot it a last time and select the intersting time points

In [ ]:
ta.rel_wave=[430,487,525,640,720,820,900,950]
plt.close('all')
ta.Plot_RAW(0,print_click_position=True)

again, the clicked points are converted to time points and we have set the intersting time points.

In [ ]:
ta.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]

Now we are ready to make an overview plot of the data

In [ ]:
ta.log_scale=False
ta.Save_Powerpoint(save_Fit=False,title='Tutorial plot')

For fitting usually the kinetic plots are a good choice to select the times I believe are a good starting point for a fit. In this process i look for all the clear decays and click on them. I also look for if there is signal before time=0 (nothing here) and if there is a signal after all is decayed (nothing here)

In [ ]:
ta.Plot_RAW(1,print_click_position=True)

Based on these clicks we create a model. In general starting with exponential decays is a good idea. Here we choose three distinct times in lines 4-6, add instrument parameter (fixed for now) in rows 8 and 9. 
Important is the line 10-12. This is a simple trick to "freeze" all parameter. This allows us to check how good are our starting parameter.
Line 14 is triggering the fit. And line 17 plotting the results

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
###-------Adding instrument parameter, here frozen---------------
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
if 1:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

In [ ]:
species=pf.Species_Spectra(ta)

In [ ]:
for key in species.keys():
    ta.Plot_RAW(0,ds=species[key])

As everything looks pretty good we simply copy paste the same code, but now set the if switch in line 9 to "0" to disable this loop and premit the optimization 

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
###-------Adding instrument parameter, here frozen---------------
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
if 0:
    for key in par.keys():
        par[key].vary=False
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

The result is refining and getting much better, but the change to the starting values is very small. So we repeat the same fit but allow the time resolution to be adjusted optimizing the instrument parameter. (line 6 and 7)

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=True)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=True)       # Allow the instrument response to adjust (False here)

ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

From this we fint and fix the instrument resolution and the arrival time of the laser.

In [ ]:
ta.mod='exponential'       # Choose a model here 'exponential' to get simple exponential decays
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output()                            # plot the fit output

A quick look on the DAS it is clear that they are the change of the spectra, to get species associated spectra we change to target analysis and the related model A->B->C by changing the model to 'consecutive' but keeping the same parameter.

In [ ]:
plt.close('all')
ta.Plot_fit_output(0) 

In [ ]:
ta.mod='consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=-0.01837,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting

plt.close('all')
ta.Plot_fit_output(0)                            # plot the fit output
ta.Plot_fit_output(4)                           # plot the fit output

the longer timepoints are very well represented, but there are still some errors in the early times. So we permit the laser arrival time, and the instrument response function free again and change the model to "full_consecutive". (In case you are in a hurry, set the vary of 't0' and 'resolution' to False. 

In [ ]:
ta.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.14,vary=True)                  
par.add('k1',value=1/2.35,vary=True)             
par.add('k2',value=1/40,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                     
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       
ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting
ta.Save_project('full_consecutive_fit.hdf5')
# saved about 45s

ta=pf.TA('full_consecutive_fit.hdf5',path=ta.path)
ta.Print_Results()
plt.close('all')
ta.Plot_fit_output(0)                            # plot the fit output
ta.Plot_fit_output(4)

This results in a 60 percent improvement of the R2 factor. But still all the DAS contain a contribution of the ground state bleach. So we are adding the ground state explicitely to try to separate the contributions. We lock the resolution and the laser arrival time "I0" to speed up the fit

In [ ]:
ta.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.100143,vary=True)                  
par.add('k1',value=1/2.496702,vary=True)             
par.add('k2',value=1/39.963222,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)       # Allow the instrument response to adjust (False here)
par.add('explicit_GS')

ta.par=par                                                     # write parameter object into file for fitting
ta.Fit_Global()                                 # trigger fitting
ta.Save_project('full_consecutive_fit_with_GS.hdf5')
# saved about 45s
reload(pf)
ta=pf.TA('full_consecutive_fit_with_GS.hdf5',path=ta.path)
ta.Print_Results()
plt.close('all')
ta.Plot_fit_output(4)                            # plot the fit output
ta.Plot_fit_output(0)

Now we can compare the time evolution of the components with the species associated spectra that were put into the data.
![Chirp](img/Intro_tutorial.png "Data_content")

It is a very imporant step to check the confidence interval, that unfortunately does take quite some time. (typically 100x the time for  a single optimization. It is generally a good idea to save the project before you do that with ta.Save_project() to not loose the prior work. The following cell shows the result of this run. The file "con_1_solved.hdf5" contains the project with the result for you to inspect.

In [ ]:
#ta.Fit_Global(confidence=0.95)
#ta.Save_project('con_1_solved.hdf5')

In [ ]:
#ta.Fit_Global(confidence=0.95)
#ta.Save_project('con_1_solved.hdf5')
#Saved 20min

ta=pf.TA('con_1_solved.hdf5',path=ta.path)
ta.filename='Solved_file.hdf5'
ta.Print_Results()

In [ ]:
# save the results
plt.close('all')
ta.Save_Powerpoint(title='Tutorial plot after Fit')

## Real data

In real data, the measured signals are not as nice and clear as we have worked with up to now.<br>
For 'con_2.SIA','con_3.SIA','con_4.SIA','con_5.SIA' 'con_6.SIA' typical disturbances were introduced. <br>
for which additional complications such as noise, chirp and crossphase modulation was added.<br>

1. Use the function "Cor_chirp" that is part of the ta object to correct the chrip in "con_2.SIA".
1. Apply the same chirp correction (either via the file name or the ta.fitcoeff to the following files. You do want to use ta.intensity range and ta.log_plot=True/False to make the development in this file visible
1. In file con_3.SIA you additionally need to adress the spectral limits using "bordercut"
1. In file con_4.SIA we have typical artifacts and Cross-Phase-Modulation. Use "ignore_time_region" to blind this out 
1. in File con_5.SIA we have to additionally reject a spectral region in which the pump laser light scattered into the detector, as is often the case if measuring e.g. nano particles.
1. In the final file con_6.SIA our initial state has some Frank condon type oscillations. Fit the data with the kinetics, and find the oscillations in the Plot_Fit_output.

In [ ]:
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')

In [ ]:
ta1=pf.TA('con_2.SIA',path=filepath)
ta1.intensity_range=0.005
ta1.log_scale=True
ta1.Plot_RAW(0)

In [ ]:
ta1.Cor_Chirp()

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.Plot_RAW(0)

In [ ]:
ta1=pf.TA('con_3.SIA',path=filepath)
ta1.Cor_Chirp(chirp_file='con_2_chirp.dat')

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.Plot_RAW(0)

In [ ]:
ta1.bordercut=[390,1150]

In [ ]:
ta1=pf.TA('con_4.SIA',path=filepath)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
plt.close('all')
ta1.Plot_RAW(0)


In [ ]:
ta1.ignore_time_region=[-0.15,0.1]

In [ ]:
ta1=pf.TA('con_5.SIA',path=filepath)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=True
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.ignore_time_region=[-0.15,0.1]
plt.close('all')
ta1.Plot_RAW(0)

In [ ]:
ta1.scattercut=[525,580]

## Lets look on some oscillations

In [ ]:
reload(pf)

In [ ]:
filepath = os.path.join(os.getcwd(), 'Data', 'Introduction')
ta1=pf.TA('con_6.SIA',path=filepath)
chirp=[-1.29781491e-11,4.72546618e-08,-6.36421133e-05,3.77396295e-02,-8.08783621e+00]
ta1.Cor_Chirp(fitcoeff=chirp)

ta1.intensity_range=0.005
ta1.log_scale=False
ta1.timelimits=[-0.2,500]
ta1.bordercut=[390,1150]
ta1.scattercut=[525,580]
ta1.rel_wave=[430,487,525,640,720,820,900,950]
ta1.rel_time=[-0.1,-0.02,0.035,0.2,0.5,2,14,22,92,160]
ta1.ignore_time_region=[-0.15,0.1]
plt.close('all')
ta1.Plot_RAW(0)

In [ ]:
#ta1.ds=ta1.ds-ta.re['AC']
ta1.mod='full_consecutive'    
par=lmfit.Parameters()                                       # create empty parameter object
par.add('k0',value=1/0.100143,vary=True)                  
par.add('k1',value=1/2.496702,vary=True)             
par.add('k2',value=1/39.963222,vary=True)   
par.add('t0',value=0,min=-2,max=2,vary=False)                       # Allow the arrival time to adjust? (False here)
par.add('resolution',value=0.086081,min=0.04,max=0.5,vary=False)
par.add('explicit_GS')
ta1.par=par
if 1:
    for key in par.keys():
        par[key].vary=False
ta1.Fit_Global()
#ta1.Save_project()
#ta1=pf.TA(ta1.filename,path=ta1.path)
plt.close('all')
ta1.Plot_fit_output(0)
ta1.Plot_fit_output(4)

In [ ]:
plt.close('all')
ta1.Plot_fit_output(1)
ta1.Plot_fit_output(2)

In [ ]:
ta2=ta1.Copy()
ta2.ds=ta1.re['AE']
ta2.intensity_range=3e-4
ta2.rel_wave=[620,700,740,800,830,860]
ta2.timelimits=[0.1,10]
ta2.Plot_RAW(0,scale_type='linear')

In [ ]:
dicten=pf.Species_Spectra(ta1)
keys=list(dicten.keys())
dicten.keys()

In [ ]:
dicten=pf.Species_Spectra(ta1)

ta3=ta1.Copy()
ta3.ds=ta1.re['A']-dicten[1]-dicten[2]-dicten['GS']

ta3.intensity_range=3e-4
ta3.timelimits=[0.1,10]
plt.close('all')
ta3.Plot_RAW([0,1],scale_type='linear')

In [ ]:
import function_library as func

In [ ]:
import function_library as func
ta2.mod=func.oscil_comp   

par=lmfit.Parameters()                                       # create empty parameter object
par.add('f0',value=1.00561,vary=True)                  
par.add('tk0',value=1/2.8725,vary=True,min=1/4,max=4)
par.add('S0',value=0.975956,vary=True ,min=0,max=1)
ta2.par=par
ta2.ignore_time_region=[-0.15,0.25]
#ta2.Fit_Global(other_optimizers='least_squares')
ta2.Fit_Global()

plt.close('all')
ta2.error_matrix_amplification=1
ta2.Plot_fit_output([0,4],scale_type='linear')

In [ ]:
#This takes about 2min
#ta2.Fit_Global(confidence_level=0.95)

ta2=pf.TA('Fitted_Oscillations_with_confidence.hdf5',path=filepath)
ta2.Print_Results()

# Why not to fit the Artifact

In [ ]:
import scipy.constants as const
import numpy as np
dicten={
    'IR-1100':{'water':1/(const.c/1.325),
               'glas':1/(const.c/1.449),
               'BBO':1/(const.c/1.654),
               'CaF':1/(const.c/1.428)},
'UV-350':{'water':1/(const.c/1.343),
          'glas':1/(const.c/1.477),
          'BBO':1/(const.c/1.707),
          'CaF':1/(const.c/1.447)}}

In [ ]:
IR=1e-3*dicten['IR-1100']['water']#+1e-3*dicten['IR-1100']['glas']
UV=1e-3*dicten['UV-350']['water']#+1e-3*dicten['UV-350']['glas']
print('%.0f fs mismatch'%((UV-IR)*1e15))
print('%.0f fs total for 35fs laser'%(np.sqrt((35e-15**2)*2+(60e-15**2))*1e15))
print('%.0f fs total for 90fs laser'%(np.sqrt((75e-15**2)*2+(60e-15**2))*1e15))

In [ ]:
IR=0.2e-3*dicten['IR-1100']['water']#+1e-3*dicten['IR-1100']['glas']
UV=0.2e-3*dicten['UV-350']['water']#+1e-3*dicten['UV-350']['glas']
print('%.0f fs mismatch'%((UV-IR)*1e15))
print('%.0f fs total for 35fs laser'%(np.sqrt((35e-15**2)*2+(12e-15**2))*1e15))